# Chapter 3. 텐서 구조체

- 딥러닝의 시작은 입력을 부동소수점으로 변환하는 것이고, 변환한 값을 담을 곳이 필요하다.
- PyTorch에서는 텐서(Tensor)라는 자료구조를 사용한다.
- 딥러닝에서의 텐서는 임의의 차원을 가진 벡터 또는 행렬의 일반화된 개념이다.
- 텐서를 다차원 배열(multi-dimensional array)이라고 볼 수 있다.



## 텐서 기초 조작

In [2]:
import torch

In [5]:
# 크기가 3인 1차원 텐서에 값을 1로 채우기
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [8]:
# 인덱스로 접근 가능
print(a[0], a[1])

tensor(1.) tensor(1.)


In [9]:
# python float로 변환
float(a[1])

1.0

In [10]:
# 특정 인덱스에 값 할당
a[2] = 2.
a

tensor([1., 1., 2.])

In [14]:
# 2차원 텐서
points = torch.tensor([[4.0, 1.0,], [5.0, 3.0], [2.0, 1.0]])
print(points)
print(points.shape)
# 스칼라
print(points[0, 1])
# 1차원 텐서
print(points[0])

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
torch.Size([3, 2])
tensor(1.)
tensor([4., 1.])


In [16]:
### 텐서 인덱싱

# 첫 번째 이후 모든 행에 대해, 모든 열이 선택됨
print(points[1:])
# 위와 같은 결과이지만, 열 표현을 명확히 하였음
print(points[1:, :])
# 첫 번째 이후 모든 행에 대해, 첫 번째 열만 선택됨
print(points[1:, 0])
# 길이가 1인 차원을 추가한다. Unsqueeze와 동일
print(points[None])

tensor([[5., 3.],
        [2., 1.]])
tensor([[5., 3.],
        [2., 1.]])
tensor([5., 2.])
tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])


## 이름이 있는 텐서

어느 차원에 어떤 데이터가 있는지 알 수 있도록 이름을 붙일 수 있다.

In [18]:
### 예시 데이터 세팅
img_t = torch.randn(3, 5, 5) # 3채널, 5x5 이미지
weights = torch.tensor([0.2126, 0.7152, 0.0722]) # RGB 가중치

In [19]:
### 배치 예시 데이터 세팅
batch_t = torch.randn(2, 3, 5, 5)

In [20]:
# RGB 채널은 img_t, batch_t 에서 항상 뒤에서 세 번째 차원에 위치한다. (-3 인덱스)
img_graph_naive = img_t.mean(-3)
batch_graph_naive = batch_t.mean(-3)
img_graph_naive.shape, batch_graph_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

`batch_t`의 경우, 배치 차원으로 인해 5x5 이미지가 2개가 있다. 따라서 shape은 (2, 5, 5)이다.

두 텐서의 Shape이 다르면 일반적으로는 연산이 불가능하다.

하지만, Shape의 일부만 다른 특정 경우에는 PyTorch가 자동으로 Shape을 맞춰 연산해준다. 이를 브로드캐스팅(broadcasting)이라고 한다.

채널의 가중치가 들어있는 weights 텐서를 `img_t`와 `batch_t`에 적용하려면, (3,) 인 weights 텐서를 (3, 1, 1)로 변환해야 한다.

이 때, unsqueeze를 사용하여 차원을 추가하고, unsqueeze를 사용하여 차원을 맞춰준다.

In [23]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
unsqueezed_weights.shape

torch.Size([3, 1, 1])

In [25]:
img_weights = img_t * unsqueezed_weights
batch_weights = batch_t * unsqueezed_weights

img_graph_weighted = img_weights.sum(-3)
batch_graph_weighted = batch_weights.sum(-3)

batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

위의 결과로 볼 수 있듯이, (3, 1, 1) 모양의 `unsqueezed_weights` 텐서가 배치 입력인 `batch_t`에 적용되었을 때, PyTorch는 자동으로 `unsqueezed_weights`를 (2, 3, 1, 1)로 변환하여 연산이 수행된 것이다.

그런데 코드가 약간 복잡해졌다. `einsum`을 사용하면 차원별로 이름을 부여할 수 있다.

In [26]:
img_graph_weighted_fancy = torch.einsum('...chw, c -> ...hw', img_t, weights)
batch_graph_weighted_fancy = torch.einsum('...chw, c -> ...hw', batch_t, weights)
batch_graph_weighted_fancy.shape

torch.Size([2, 5, 5])

그러나 이 방법도 많은 기호가 복잡하게 사용되고 있다. 보다 직관적인 방법으로 `named_tensor`를 사용할 수 있다.



In [31]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channel',])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channel',))

In [32]:
# 나중에 이름을 지정하려면 refine_names를 사용한다. 텐서 접근 시 생략 부호 ...를 사용하면 다른 차원은 건드리지 않는다.
img_named = img_t.refine_names(..., 'channel', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channel', 'rows', 'columns')

print("img named: ", img_named.shape, img_named.names)
print("batch named: ", batch_named.shape, batch_named.names)

img named:  torch.Size([3, 5, 5]) ('channel', 'rows', 'columns')
batch named:  torch.Size([2, 3, 5, 5]) (None, 'channel', 'rows', 'columns')


텐서 간의 연산은 각 차원의 크기가 같거나, 한쪽이 1이어서 다른 쪽으로 브로드캐스팅이 가능해야 한다.
이때, 이름이 지정되어 있다면 PyTorch는 이름이 일치하는 차원끼리 알아서 확인해준다.

In [34]:
weights_aligned = weights_named.align_as(img_named)
# 'channel' 만 있던 weights_named가 img_named의 'channel' 차원과 일치하도록 확장된다.
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channel', 'rows', 'columns'))

In [41]:
# sum 처럼 차원 인수를 허용하는 함수들은 이름이 붙은 차원도 받아들인다.
gray_named = (img_named * weights_aligned).sum('channel')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [39]:
# 이름 있는 텐서를 사용한다고 차원을 자동으로 정렬해주지는 않는다.
try:
    gray_named = (img_named[..., :3] * weights_named).sum('channel')
except Exception as e:
    print("Error:", e)

Error: Error when attempting to broadcast dims ['channel', 'rows', 'columns'] and dims ['channel']: dim 'columns' and dim 'channel' are at the same position from the right but do not match.


In [42]:
# 이름 있는 텐서를 사용하는 연산을 함수 밖에서도 사용하려면, 차원 이름에 None을 넣어 이름 없는 텐서를 만든다.
gray_named = (img_named * weights_aligned).sum('channel')
graph_plain = gray_named.rename(None)
graph_plain.shape, graph_plain.names

(torch.Size([5, 5]), (None, None))

좀 더 부연설명하자면 이름 있는 텐서는 일부 연산에서만 동작하며,
대부분의 PyTorch/외부 함수는 이름 없는 텐서만 지원하기 때문에 호환성을 위해 .rename(None)으로 이름을 제거하는 것이다.

## 텐서의 데이터 타입

### 파이썬 표준 타입을 사용하지 않는 이유

텐서는 다양한 데이터 타입을 가진다. 그러나 파이썬 타입은 사용하지 않는다. 그 이유는 다음과 같다.

- 파이썬에서 숫자는 박싱(boxing)되어 객체로 다뤄진다. 따라서 수 백만 개 이상을 다뤄야 하는 딥러닝에서는 비효율적이다.
- 파이썬의 리스트는 단순히 연속된 객체의 컬렉션이다. 이로 인해, 다음과 같은 비효율이 발생한다.
  - 두 벡터의 내적, 합 등의 연산을 효율적으로 수행할 수 있는 연산이 지원되지 않는다.
  - 리스트에 들어있는 데이터를 메모리에 최적화하여 배치할 방법이 없다.
  - 리스트는 기본적으로 단일 차원만 지원하며, 중첩된 리스트로 다차원 배열을 구현할 수 있지만, 여전히 비효율적이다.
- 파이썬 인터프리터는 최적화를 거친 컴파일된 코드보다 느리다.

이러한 이유로 데이터과학 라이브러리는 NumPy에 의존하거나 PyTorch 텐처처럼 전용 데이터 구조를 만든 후 숫자 데이터 연산은 저수준 언어로 효율을 높이도록 구현한다.
성능 최적화를 위해 텐서 내의 모든 객체는 같은 타입의 숫자여야 하고, PyTorch는 실행 중에 이런 숫자 타입을 계속 추적해야 한다.

### 데이터 타입

`tensor`, `zeros`, `ones` 등의 함수는 `dtype` 인수를 사용하여 텐서의 숫자 타입을 지정할 수 있다.

`dtype` 인자는 표준 NumPy 타입과 거의 동일하다. `dtype` 인자에 지정할 수 있는 타입은 다음과 같다.

| 타입                              | 설명           |
|---------------------------------|--------------|
| `torch.float32`, `torch.float`  | 32비트 부동소수점   |
| `torch.float64`, `torch.double` | 64비트 부동소수점   |
| `torch.float16`, `torch.half`   | 16비트 부동소수점   |
| `torch.int8`                    | 8비트 정수       |
| `torch.uint8`                   | 8비트 부호 없는 정수 |
| `torch.int16`, `torch.short`    | 16비트 정수      |
| `torch.int32`, `torch.int`      | 32비트 정수      |
| `torch.int64`, `torch.long`      | 64비트 정수      |
| `torch.bool`                    | 불리언 값       |

### 가장 많이 사용되는 데이터 타입

- 신경망 연산
  - 신경망 연산은 대부분 32비트 부동소수점으로 수행된다.
  - 배정밀도 64비트를 사용해도 모델 정확도가 크게 향상되지 않지만 더 많은 메모리를 사용하고 더 느리게 동작한다.
  -16비트 반정밀도 부동소수점은 최신 GPU에서 대부분 지원되며, 정확도를 희생해서 신경망이 차지하는 공간을 줄일 수 있다.
- 인덱스
  - 텐서는 다른 텐서에 대한 인덱스로 사용할 수 있다.
  - 이때, PyTorch는 인덱싱용 텐서를 64비트 정수 데이터 타입으로 간주한다.
- bool
  - `points > 1.0` 과 같은 predicate는 텐서 내 각각의 요소가 이 조건을 만족하는지 알려주는 bool 텐서를 만든다.


In [44]:
# bool 텐서 예시
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
mask = points > 1.0
mask

tensor([[ True, False],
        [ True,  True],
        [ True, False]])

In [46]:
### 텐서 타입 캐스팅
# dtype 메소드를 이용한 캐스팅
double_points = torch.zeros(10, 2).double()
print(torch.zeros(10, 2).dtype, double_points.dtype)

# to 메소드를 이용한 캐스팅
short_points = torch.ones(10, 2).to(dtype=torch.short)
print(torch.ones(10, 2).dtype, short_points.dtype)

# 여러 타입을 가진 입력들이 연산을 거치며 서로 섞일 때 자동으로 제일 큰 타입으로 만들어진다.
points_64 = torch.rand(5, dtype=torch.double)
points_short = points_64.to(dtype=torch.short)
points_64 * points_short

torch.float32 torch.float64
torch.float32 torch.int16


tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

## 메모리 관점에서의 텐서

- 텐서 내부의 값은 `torch.Storage` 인스턴스로 관리하며, 연속적인 메모리 블록에 저장된다.
- 텐서 객체는 저장 공간을 추상화한 `Storage` 객체에 대한 뷰 (view) 역할을 하며, 오프셋을 사용해 공간의 임의 위치에 접근하거나 특정 차원의 크기를 단위로 해서 접근할 수 있다.



In [47]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
# 텐서가 세 개의 행과 두 개의 열을 가지고 있지만, 실제로는 크기가 6인 배열 공간이다.
points.storage()

/var/folders/4l/lb4d41pn56d4wnwsr9rxl20c0000gn/T/ipykernel_75000/3297796694.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  points.storage()


 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [48]:
points.storage()[2]  # 2번째 인덱스에 있는 값

5.0

## 텐서 메타데이터 - 사이즈, 오프셋, 스트라이드

저장 공간을 인덱스로 접근할 때, 사이즈와 오프셋 그리고 스트라이드에 의존한다.

- 사이즈(NumPy의 Shape와 같음): 차원별 요소의 수를 표시한 튜플
- 오프셋: 저장 공간에 대한 오프셋은 색인 0의 요소가 저장된 위치를 나타낸다.
- 스트라이드: 각 차원에서 다음 요소를 가리키고 싶을 때, 실제 저장 공간 상에서 몇 개의 요소를 건너뛰어야 하는지를 알려준다.

In [56]:
x33_t = torch.tensor([[5.0, 7.0, 4.0], [1.0, 3.0, 2.0], [7.0, 3.0, 8.0]])
print(x33_t.shape, x33_t.stride(), x33_t.storage_offset())

x33_t[1][1] # tensor(3.) 이 나와야 한다. 0 + 3 * 1 + 1 * 1이 적용되었을 것이다.
# 즉, 스토리지 오브젝트 상에서는 storage_offset() + stride[0] * 1 + stride[1] * 1 인덱스로 접근한 것이다.

torch.Size([3, 3]) (3, 1) 0


tensor(3.)

In [51]:
x33_t[1], x33_t[1].shape, x33_t[1].stride(), x33_t[1].storage_offset()

(tensor([1., 3., 2.]), torch.Size([3]), (1,), 3)

이러한 방식 덕분에 텐서의 전치(Transpose)나 일부만으로 더 작은 텐서를 만드는 데 필요한 연산을 효율적으로 수행할 수 있다.

In [4]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print(points)

points_t = points.t()  # 전치
print(points_t)

points.storage().data_ptr() == points_t.storage().data_ptr()

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])
tensor([[4., 5., 2.],
        [1., 3., 1.]])


True

코드가 책과는 다른데, `storage()` 구현이 바뀌었는지 스토리지 인스턴스의 `id` 로 비교하면 안된다. `data_ptr()`를 사용하면 실제 메모리 주소를 알 수 있다.

`data_ptr()` 의 공식 문서 설명: Returns the address of the first element of self tensor.

## 인접한 텐서

가장 오른쪽 차원에서부터 증가되는 형태로 저장소에 값이 펼쳐진 텐서는 contiguous로 정의된다.

인접한 텐서는 데이터 지역성 관점에서 CPU 메모리 접근 효율이 좋다.

In [58]:
points.is_contiguous()

True

In [59]:
points_t.is_contiguous() # 전치된 텐서는 인접하지 않음

False

In [62]:
print(points_t.stride(), points_t.storage())
points_t_cont = points_t.contiguous()
points_t_cont.stride(), points_t_cont.storage() # 새 저장 공간을 사용

(1, 2)  4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]


((3, 1),
  4.0
  5.0
  2.0
  1.0
  3.0
  1.0
 [torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6])

## 텐서를 GPU로 옮기기

지금까지는 CPU 메모리에서 텐서를 다루었다. PyTorch는 GPU를 지원하며, GPU로 텐서를 옮길 수 있다.

In [ ]:
# 생성자에서 device 인자를 사용하여 GPU에 텐서를 생성할 수 있다.
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device='cuda')

In [ ]:
# 또는, to 메소드를 사용하여 CPU 텐서를 GPU로 옮길 수 있다.
points_gpu = points.to(device='cuda')
# GPU가 둘 이상인 환경에서 특정 GPU에서 작업을 수행하게 만들 수 있다.
points_gpu = points.to(device='cuda:0')  # 첫 번째 GPU로 옮김
points_gpu = points.cuda(0) # 위와 동일한 결과

## NumPy 호환

PyTorch 텐서는 NumPy와 호환된다. Zero-copy 수준의 상호 변환이 가능한데, 이는 파이썬 버퍼 프로토콜 덕분이다.

## 연습문제

In [64]:
### 1. list(range(9))로부터 텐서를 만들어라. 사이즈, 오프셋, 스트라이드는 얼마일지도 계산해보자.
# 1차원 배열을 텐서로 만들면 1차원 텐서가 된다.
# 이로부터 사이즈는 Tensor([9]) 이고, 오프셋은 1이며 스트라이드는 (1,) 일 것이다.

a = torch.tensor(list(range(9)))
a.size(), a.storage_offset(), a.stride()

(torch.Size([9]), 0, (1,))

In [66]:
### 1-a. b = a.view(3, 3)으로 텐서를 만들어라. View의 역할은 무엇인가? a와 b 가 같은 공간을 가리키고 있는지 확인해보자.
# View는 텐서를 다른 형태로 볼 수 있도록 해준다. 이 때, 원본 텐서의 저장소를 이용한다.
b = a.view(3, 3)
b_storage = b.storage()
a_storage = a.storage()
print(b)
id(a_storage), id(b_storage)

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])


(5129082192, 5128994896)

In [68]:
### 1-b. c = b[1:, 1:]로 텐서를 만들고 사이즈, 오프셋, 스트라이드는 얼마일지 계산해보라.
# b 텐서에서 첫 번째 행 이후와 첫 번째 열 이후를 가리키므로, Tensor([[4, 5], [7, 8]])이 된다.
# 사이즈는 torch.Size([2, 2]) 이고, 오프셋은 4, 스트라이드는 (3, 1) 이다.
c = b[1:, 1:]
print(c)
c.size(), c.storage_offset(), c.stride()

tensor([[4, 5],
        [7, 8]])


(torch.Size([2, 2]), 4, (3, 1))

문제 2. 코사인이나 제곱근 같은 수학 연산을 하나 골라라.  
동일한 역할을 하는 함수를 torch 라이브러리에서 찾을 수 있을까?

->
수학 연산과 관련된 문서는 https://docs.pytorch.org/docs/stable/torch.html#math-operations 이다.

나는 표준편차를 계산하는 [`std`](https://docs.pytorch.org/docs/stable/generated/torch.std.html#torch.std) 함수를 골랐다. 이 연산은 Reduction Ops에 해당한다.



In [70]:
### 2-a. 텐서 a에 대해 해당 함수를 요소 단위로 실행해보라. 왜 오류가 발생할까?
try:
    a.std()
except Exception as e:
    print("Error:", e)

# std는 floating point와 complex dtypes 만 지원한다.

Error: std and var only support floating point and complex dtypes


In [71]:
### 2.b. 동작시키려면 어떤 연산이 필요할까?

# a를 float로 변환한 후, std를 실행하면 된다.
a_float = a.to(dtype=torch.float)
a_float.std()

tensor(2.7386)

In [73]:
### 2.c. 해당 연산을 추가 공간을 사용하지 않고 실행하는 함수가 있을까?
# PyTorch에서 to()나 float() 등의 dtype 변환 연산은 항상 새로운 메모리를 사용하는 non-inplace 연산이다.
# 따라서 추가 공간없이 표준 편차를 계산할 수 없다.